In [1]:
#import the necessary modules
!pip install folium


In [2]:
import folium
import pandas as pd
import requests
import io
from io import StringIO
from math import sin, cos, sqrt, atan2, radians

In [3]:
#import folium MarkerCluster plugin, MousePosition plugin, DivIcon plugin
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon 

In [4]:
#Mark all launch Sites on a map
world_map = folium.Map()

In [5]:
#download and read "spacex_launch_geo.csv"
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = requests.get(URL)
spacex_df = pd.read_csv(StringIO(resp.content.decode('utf-8')))
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [6]:
#look at coordinates of each site and select relevant sub-columns: Launch Site, Lat(Latitude), Long(Longitude), class
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index = False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df.head()

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Initialize center as Nasa Johnson Space Center at Houston
nasa_coordinate = [29.559684888503615, -95.0830971930759]  # Corrected longitude
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a circle around NASA Johnson Space Center
circle = folium.Circle(
    location=nasa_coordinate,
    radius=1000,
    color='red',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a marker with a custom icon and popup
marker = folium.Marker(
    location=nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)

# Add the circle and marker to the map
site_map.add_child(circle)
site_map.add_child(marker)                                                                                      

In [9]:
#initialize map
site_map = folium.Map(location = nasa_coordinate, zoom_start = 5)
#add circle for each launch sites
for index, row in launch_sites_df.iterrows():
    coordinate = (row['Lat'], row['Long'])
    folium.Circle(
        coordinate, 
        radius = 1000, 
        color = 'blue', 
        fill = True
    ).add_child(folium.Popup(spacex_df['Launch Site'])).add_to(site_map)
    folium.Marker(
        coordinate,
        icon = DivIcon(
            icon_size = (20, 20), 
            icon_anchor = (0, 0), 
            html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],  
            )
        ).add_to(site_map)
site_map

All launch sites are somewhat close to both the Equator line and the coast. Launching from the Equator can ensure the rocket captures the maximum amount of the Earth's rotational speed. Launching along the coast is a good safety choice and means that if the rocket fails, there is a significantly smaller chance of it landing in land. 

In [10]:
# differentiate between success and failed launches for each site on map
spacex_df.tail(10)

#class 1 = success, class 0 = fail
#create function to append either red or green to spacex_df
marker_cluster = MarkerCluster()
def marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(marker_color)
spacex_df.tail(15)

#add marker_cluster to current site_map
site_map.add_child(marker_cluster)

#create a marker object with coordinate and customize icon to indicate success or fail
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    folium.Marker(
        coordinate,
        icon = folium.Icon(
            color = 'white',
            icon_color = row['marker_color']
        )
    ).add_to(marker_cluster)
site_map

In [11]:
#calculate distances between launch sites to its proximities
#add mouse position to get coordinate for mouse on the map
formatter = 'function(num) {return L.Util.formatNum(num, 5);};'
mouse_position = MousePosition(
    position = 'topright', 
    separator = 'Long: ',
    empty_string = 'NaN',
    lng_first = False, 
    num_digits = 20,
    prefix = 'Lat:', 
    lat_formatter = formatter, 
    lng_formatter = formatter, 
)

site_map.add_child(mouse_position)
site_map

In [12]:
def cal_distance(lat1, lon1, lat2, lon2):
    R = 6373.0 #radius of earth in Km
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dis_lon = lon2 - lon1
    dis_lat = lat2 - lat1
    
    a = sin(dis_lat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dis_lon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    distance = R * c
    return distance

In [13]:
# find coordinate of the closet coastline
launch_site_lat = 28.563197
launch_site_lon = -80.576820
coastline_lat = 28.56334
coastline_lon = -80.56799
distance_coastline = cal_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print('distance from coastline is: ', distance_coastline,' km')

distance from coastline is:  0.8627671182499878  km


In [14]:
#Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# draw a line between launch site and selected coastline point
distance_marker = folium.Marker(
   [coastline_lat, coastline_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#FFFF00;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
site_map.add_child(distance_marker)

coordinates = [[launch_site_lat,launch_site_lon],[coastline_lat,coastline_lon]]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

This allows the user to draw a line to the closest city, railway, highway, etc. They just need to find the mouse position of the desired structure

In [15]:
#find the closest city, railway, highway and record the distance
closest_highway = 28.56335, -80.57085
closest_railroad = 28.57206, -80.58525
closest_city = 28.10473, -80.64531

distance_highway = cal_distance(launch_site_lat, launch_site_lon, closest_highway[0], closest_highway[1])
print('distance of nearest highway: ',distance_highway, 'km')
distance_railroad = cal_distance(launch_site_lat, launch_site_lon, closest_railroad[0], closest_railroad[1])
print('distance of nearest railroad: ',distance_railroad, 'km')
distance_city = cal_distance(launch_site_lat, launch_site_lon, closest_city[0], closest_city[1])
print('distance of nearest city: ',distance_city, 'km')

distance of nearest highway:  0.5834695366934144 km
distance of nearest railroad:  1.2845344718142522 km
distance of nearest city:  51.43416999517233 km


In [16]:
#draw a line between nearest highway, railway, etc to the launch site

# closest highway marker
distance_marker = folium.Marker(
   closest_highway,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#99FF33;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
       )
   )
site_map.add_child(distance_marker)

# closest highway line
coordinates = [[launch_site_lat,launch_site_lon],closest_highway]
lines=folium.PolyLine(locations=coordinates, weight=2)
site_map.add_child(lines)

# closest railroad marker
distance_marker = folium.Marker(
   closest_railroad,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#3399FF;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railroad),
       )
   )
site_map.add_child(distance_marker)
# closest railroad line
coordinates = [[launch_site_lat,launch_site_lon],closest_railroad]
lines=folium.PolyLine(locations=coordinates, weight=2)
site_map.add_child(lines)

# closest city marker
distance_marker = folium.Marker(
   closest_city,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#FF66CC;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
       )
   )
site_map.add_child(distance_marker)
# closest city line
coordinates = [[launch_site_lat,launch_site_lon],closest_city]
lines=folium.PolyLine(locations=coordinates, weight=2)
site_map.add_child(lines)